In [8]:
import pandas as pd
import numpy as np

In [42]:
data = ('Datasets/kc_house_data.csv')


def load_data():
    columns = {
        'id': 'ID',
        'date': 'data',
        'price': 'preço',
        'bedrooms': 'quartos',
        'bathrooms': 'banheiros',
        'sqft_living': 'area M2',
        'floors': 'andares',
        'waterfront': 'beira_mar',
        'yr_built': 'ano_construção',
        'zipcode': 'CEP',
        'lat': 'latitude',
        'long': 'longitude'
    }
    dataframe = pd.read_csv(data)
    dataframe = dataframe.rename(columns=columns)
    
    dataframe['data'] = pd.to_datetime(dataframe['data'])
    dataframe['banheiros'] = np.int64(round(dataframe['banheiros'] - 0.3))
    dataframe['andares'] = np.int64(round(dataframe['andares'] - 0.3))
    dataframe['beira_mar'] = dataframe['beira_mar'].apply(lambda x: 'SIM' if x == 1 else 'NÃO')
    dataframe['level'] = dataframe['preço'].apply(lambda x: 1 if (x >= 75000) & (x < 321950) else
                                                            2 if (x >= 321950) & (x < 568900) else
                                                            3 if (x >= 568900) & (x < 815850) else
                                                            4 if (x >= 815850) & (x < 1062800) else
                                                            5)
    dataframe['idade'] = dataframe["data"].dt.year - dataframe["ano_construção"]
    dataframe = dataframe[list(columns.values())]
    
    return dataframe


data = load_data()
data.head()

,ID,data,preço,quartos,banheiros,area M2,andares,beira_mar,ano_construção,CEP,latitude,longitude
0,7129300520,2014-10-13,221900.00000,3,1,1180,1,NÃO,1955,98178,47.51120,-122.25700
1,6414100192,2014-12-09,538000.00000,3,2,2570,2,NÃO,1951,98125,47.72100,-122.31900
2,5631500400,2015-02-25,180000.00000,2,1,770,1,NÃO,1933,98028,47.73790,-122.23300
3,2487200875,2014-12-09,604000.00000,4,3,1960,1,NÃO,1965,98136,47.52080,-122.39300
4,1954400510,2015-02-18,510000.00000,3,2,1680,1,NÃO,1987,98074,47.61680,-122.04500
